# importacoes

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from missingpy import MissForest
from sklearn.preprocessing import LabelEncoder

from IPython.display import display

c:\users\vinic\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def printa_df(df):
    
    print(df.shape)
    display(df.head())
    
pd.set_option('display.max_columns', None)

In [3]:
df_credito = pd.read_csv('../../../../base_de_dados/credit_card_balance.csv')

printa_df(df_credito)

(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [4]:
df_credito = df_credito[df_credito['MONTHS_BALANCE'] >= -6]

In [5]:
le = LabelEncoder()

df_credito['NAME_CONTRACT_STATUS'] = le.fit_transform(df_credito['NAME_CONTRACT_STATUS'])

# analise

In [6]:
df_credito.isna().sum()/len(df_credito)*100

SK_ID_PREV                     0.000000
SK_ID_CURR                     0.000000
MONTHS_BALANCE                 0.000000
AMT_BALANCE                    0.000000
AMT_CREDIT_LIMIT_ACTUAL        0.000000
AMT_DRAWINGS_ATM_CURRENT      28.420027
AMT_DRAWINGS_CURRENT           0.000000
AMT_DRAWINGS_OTHER_CURRENT    28.420027
AMT_DRAWINGS_POS_CURRENT      28.420027
AMT_INST_MIN_REGULARITY        0.000000
AMT_PAYMENT_CURRENT           28.875995
AMT_PAYMENT_TOTAL_CURRENT      0.000000
AMT_RECEIVABLE_PRINCIPAL       0.000000
AMT_RECIVABLE                  0.000000
AMT_TOTAL_RECEIVABLE           0.000000
CNT_DRAWINGS_ATM_CURRENT      28.420027
CNT_DRAWINGS_CURRENT           0.000000
CNT_DRAWINGS_OTHER_CURRENT    28.420027
CNT_DRAWINGS_POS_CURRENT      28.420027
CNT_INSTALMENT_MATURE_CUM      0.000000
NAME_CONTRACT_STATUS           0.000000
SK_DPD                         0.000000
SK_DPD_DEF                     0.000000
dtype: float64

In [7]:
x = df_credito.drop(['SK_ID_PREV', 'SK_ID_CURR'], axis=1)
mf = MissForest(max_depth=10, max_iter=3, n_estimators=10, max_features=0.7, min_samples_split=10,
                min_impurity_decrease=0.01)

mf.fit(x)

MissForest(max_depth=10, max_features=0.7, max_iter=3,
           min_impurity_decrease=0.01, min_samples_split=10, n_estimators=10)

In [8]:
transform = mf.transform(x)

Iteration: 0
Iteration: 1
Iteration: 2


In [9]:
df_credito[x.columns] = transform

In [10]:
df_credito['Sacou_mais_dinhero_que_o_limite'] = df_credito['AMT_CREDIT_LIMIT_ACTUAL'] - df_credito['AMT_DRAWINGS_CURRENT']
df_credito['Comprou_mais_que_o_limite'] = df_credito['AMT_CREDIT_LIMIT_ACTUAL'] - df_credito['AMT_DRAWINGS_POS_CURRENT']
df_credito['Pagou_menos_que_o_valor'] = df_credito['AMT_PAYMENT_TOTAL_CURRENT'] - df_credito['AMT_INST_MIN_REGULARITY']

In [11]:
df_credito

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF,Sacou_mais_dinhero_que_o_limite,Comprou_mais_que_o_limite,Pagou_menos_que_o_valor
0,2562384,378907,-6.0,56.970,135000.0,0.000000,877.5,0.0,877.500000,1700.325,1800.000000,1800.0,0.000,0.000,0.000,0.000000,1.0,0.000000,1.000000,35.0,0.0,0.0,0.0,134122.5,134122.500000,99.675
1,2582071,363914,-1.0,63975.555,45000.0,2250.000000,2250.0,0.0,0.000000,2250.000,2250.000000,2250.0,60175.080,64875.555,64875.555,1.000000,1.0,0.000000,0.000000,69.0,0.0,0.0,0.0,42750.0,45000.000000,0.000
3,1389973,337855,-4.0,236572.110,225000.0,2250.000000,2250.0,0.0,0.000000,11795.760,11925.000000,11925.0,224949.285,233048.970,233048.970,1.000000,1.0,0.000000,0.000000,10.0,0.0,0.0,0.0,222750.0,225000.000000,129.240
4,1891521,126868,-1.0,453919.455,450000.0,0.000000,11547.0,0.0,11547.000000,22924.890,27000.000000,27000.0,443044.395,453919.455,453919.455,0.000000,1.0,0.000000,1.000000,101.0,0.0,0.0,0.0,438453.0,438453.000000,4075.110
6,1079071,171320,-6.0,353451.645,585000.0,67500.000000,67500.0,0.0,0.000000,14684.175,15750.000000,15750.0,345433.860,351881.145,351881.145,1.000000,1.0,0.000000,0.000000,6.0,0.0,0.0,0.0,517500.0,585000.000000,1065.825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3831214,2286534,116204,-2.0,0.000,180000.0,0.000000,0.0,0.0,0.000000,0.000,0.000000,0.0,0.000,0.000,0.000,0.000000,0.0,0.000000,0.000000,55.0,1.0,0.0,0.0,180000.0,180000.000000,0.000
3831216,2567648,184352,-3.0,0.000,45000.0,0.000000,0.0,0.0,0.000000,0.000,2190.015000,0.0,0.000,0.000,0.000,0.000000,0.0,0.000000,0.000000,63.0,0.0,0.0,0.0,45000.0,45000.000000,0.000
3831218,2064327,165054,-6.0,0.000,0.0,1005.893364,0.0,0.0,347.615673,0.000,828.041146,0.0,0.000,0.000,0.000,0.692703,0.0,0.002795,1.306068,0.0,0.0,0.0,0.0,0.0,-347.615673,0.000
3834259,1151858,199490,-1.0,89642.070,90000.0,2250.000000,2250.0,0.0,0.000000,4500.000,4500.000000,4500.0,85008.015,89642.070,89642.070,1.000000,1.0,0.000000,0.000000,80.0,0.0,0.0,0.0,87750.0,90000.000000,0.000


In [13]:
df_credito = df_credito.groupby('SK_ID_CURR').mean().reset_index()

df_credito.to_csv('../../../../base_de_dados/features_credito.csv', index=False)